In [1]:
import utils
import numpy as np
import matplotlib.pyplot as plt
import params
import data_loader

In [2]:
x_train, y_train, x_test, y_test = data_loader.get_train_test()

Load .edf -> .\data\PSGData1_Hang7\20190917-T3-93135.edf
Extracting EDF parameters from E:\ZJU_Research\SR_ZJUPH_PSG\data\PSGData1_Hang7\20190917-T3-93135.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


E:\ZJU_Research\SR_ZJUPH_PSG\utils.py:27: RuntimeWarning: Physical range is not defined in following channels:
Thor, Abdo
  raw_train = mne.io.read_raw_edf(path)


1186 36441088
Normalizing ... Current Dimension -> 0
Normalizing ... Current Dimension -> 1
Normalizing ... Current Dimension -> 2
Normalizing ... Current Dimension -> 3
Normalizing ... Current Dimension -> 4
Normalizing ... Current Dimension -> 5
Normalizing ... Current Dimension -> 6
Normalizing ... Current Dimension -> 7
Normalizing ... Current Dimension -> 8
Normalizing ... Current Dimension -> 9
Normalizing ... Current Dimension -> 10
Normalizing ... Current Dimension -> 11
Normalizing ... Current Dimension -> 12
Normalizing ... Current Dimension -> 13
Normalizing ... Current Dimension -> 14
Normalizing ... Current Dimension -> 15
Normalizing ... Current Dimension -> 16
Normalizing ... Current Dimension -> 17
Normalizing ... Current Dimension -> 18
Normalizing ... Current Dimension -> 19
Normalizing ... Current Dimension -> 20
Return data, np.std(data) is too Low! len data -> 36441088
Normalizing ... Current Dimension -> 21
Normalizing ... Current Dimension -> 22
Return data, np.s

E:\ZJU_Research\SR_ZJUPH_PSG\utils.py:27: RuntimeWarning: Physical range is not defined in following channels:
Thor, Abdo
  raw_train = mne.io.read_raw_edf(path)


1168 35885056
Normalizing ... Current Dimension -> 0
Normalizing ... Current Dimension -> 1
Normalizing ... Current Dimension -> 2
Normalizing ... Current Dimension -> 3
Normalizing ... Current Dimension -> 4
Normalizing ... Current Dimension -> 5
Normalizing ... Current Dimension -> 6
Normalizing ... Current Dimension -> 7
Normalizing ... Current Dimension -> 8
Normalizing ... Current Dimension -> 9
Normalizing ... Current Dimension -> 10
Normalizing ... Current Dimension -> 11
Normalizing ... Current Dimension -> 12
Normalizing ... Current Dimension -> 13
Normalizing ... Current Dimension -> 14
Normalizing ... Current Dimension -> 15
Normalizing ... Current Dimension -> 16
Normalizing ... Current Dimension -> 17
Normalizing ... Current Dimension -> 18
Normalizing ... Current Dimension -> 19
Normalizing ... Current Dimension -> 20
Normalizing ... Current Dimension -> 21
Normalizing ... Current Dimension -> 22
Normalization Done!
Generating Train&Test ... Current Index -> 0
Generating 

MemoryError: Unable to allocate 6.15 GiB for an array with shape (1168, 23, 30726) and data type float64

In [ ]:
!pip install visualkeras

In [ ]:
!conda install -c conda-forge ann -y
!conda install -c conda-forge ann_visualizer -y
!conda install graphviz -y
!conda install -c anaconda python-graphviz -y
!conda install -c anaconda pydot -y

In [ ]:
from models import cnn_1, cnn_2, cnn_lstm, cnn_crf, resnet_101v2, efficient_net_v2l, vgg_19
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, accuracy_score, classification_report
from glob import glob
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
from ann_visualizer.visualize import ann_viz
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import tensorboard
import datetime
from tensorflow.keras.utils import plot_model
import visualkeras

In [ ]:
MODEL_LIST = []

In [ ]:
MODEL_LIST.append(cnn_1())

In [ ]:
MODEL_LIST.append(cnn_2())

In [ ]:
MODEL_LIST.append(cnn_lstm())

In [ ]:
MODEL_LIST.append(cnn_crf())

In [ ]:
NAME_LIST = ["cnn_1", "cnn_2", "cnn_lstm", "cnn_crf"]

In [ ]:
HIST_LIST, TRAINED_MODEL = [], []

In [ ]:
plot_model(MODEL_LIST[0], to_file="./images/" + NAME_LIST[0] + ".png", show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(MODEL_LIST[1], to_file="./images/" + NAME_LIST[1] + ".png", show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(MODEL_LIST[2], to_file="./images/" + NAME_LIST[2] + ".png", show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(MODEL_LIST[3], to_file="./images/" + NAME_LIST[3] + ".png", show_shapes=True, show_layer_names=True)

In [ ]:
DISABLE_GPU = False
if DISABLE_GPU:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
def get_callbacks(file_path, model_name):
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_acc", mode="max", patience=20, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=5, verbose=2)
    log_dir = "logs/" + str(model_name) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tfboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
    callbacks_list = [checkpoint, early, redonplat, tfboard]

In [ ]:
def train_model(model, name):
    file_path = './models/' + name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5'
    callbacks_list = get_callbacks(file_path, name)
    opt = Adam(learning_rate=params.LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit((x_train, y_train),
                    epochs=params.EPOCH_NUM,
                    batch_size=params.BATCH_SIZE,
                    validation_data=(x_test, y_test),
                    verbose=params.VERBOSE,
                    callbacks=callbacks_list)
    model = load_model(file_path)
    test_loss, test_acc = model.evaluate((x_test, y_test), steps=len(y_test), verbose=1)
    print('Loss: ', test_loss)
    print('Accuracy: ', test_acc) 
    return history, model

In [ ]:
for i in range(len(MODEL_LIST)):
    history, trained_model = train_model(MODEL_LIST[i], NAME_LIST[i])
    HIST_LIST.append(history)
    TRAINED_MODEL.append(trained_model)

In [ ]:
def tr_plot(tr_data, start_epoch):
    #Plot the training and validation data
    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout    
    plt.show()

In [ ]:
def confusion_matrix_plot(matrix, classes, name):
    plt.figure(figsize=(12,10))
    cmap = "YlGnBu"
    ax= plt.subplot()
    sns.heatmap(matrix, annot=True, fmt='g', ax=ax, cmap=cmap);  #annot=True to annotate cells, ftm='g' to disable scientific notation
    plt.savefig('./images/' + name + 'con_mat.png')
    # labels, title and ticks
    ax.set_xlabel('Predicted labels');
    ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(classes); 
    ax.yaxis.set_ticklabels(classes[::-1]);
    plt.show()

In [ ]:
def cal_score(model, name, plot=True, labels=[0, 1, 2, 3, 4]):
    matrix = confusion_matrix(predictions, labels)
    print(matrix)
    print('\n')

    f1 = f1_score(predictions, labels, average='weighted')
    print(f'F1 Score: {f1}')
    print('\n')
    
    print(classification_report(predictions, labels, target_names=classes))
    
    if plot:
        confusion_matrix_plot(matrix, labels, name)

In [ ]:
def plot_and_f1(model, history, name):
    cal_score(model, name, plot=True)
    tr_plot(history, 0)
    print("==" * 80)

In [ ]:
for i in range(len(HIST_LIST)):
    plot_and_f1(TRAINED_MODEL[i], HIST_LIST[i], NAME_LIST[i])